<a href="https://colab.research.google.com/github/Priyo-prog/Artificial-Intelligence-Projects/blob/main/Emotion%20AI/emotion_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction to Emotion AI**

* Artificial Emotional Intelligence or Emotion AI is a branch of AI that allows computers to understand human non-verbal cues such as body language and facial expression.

